In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from IPython.display import display

In [18]:
data = pd.read_csv("covtype.data",header=None)
data.columns = ["Elevation","Aspect","Slope","HDH","VDH","HDR","Hillshade_9am","Hillshade_Noon","Hillshade_3pm","HDFP","Rawah_Wilderness_Area","Neota_Wilderness_Area","Comanche_Peak_Wilderness_Area","Cache_la_Poudre_Wilderness_Area","Soil_Type_USFS2702","Soil_Type_USFS2703","Soil_Type_USFS2704","Soil_Type_USFS2705","Soil_Type_USFS2706","Soil_Type_USFS2717","Soil_Type_USFS3501","Soil_Type_USFS3502","Soil_Type_USFS4201","Soil_Type_USFS4703","Soil_Type_USFS4704","Soil_Type_USFS4744","Soil_Type_USFS4758","Soil_Type_USFS5101","Soil_Type_USFS5151","Soil_Type_USFS6101","Soil_Type_USFS6102","Soil_Type_USFS6731","Soil_Type_USFS7101","Soil_Type_USFS7102","Soil_Type_USFS7103","Soil_Type_USFS7201","Soil_Type_USFS7202","Soil_Type_USFS7700","Soil_Type_USFS7701","Soil_Type_USFS7702","Soil_Type_USFS7709","Soil_Type_USFS7710","Soil_Type_USFS7745","Soil_Type_USFS7746","Soil_Type_USFS7755","Soil_Type_USFS7756","Soil_Type_USFS7757","Soil_Type_USFS7790","Soil_Type_USFS8703","Soil_Type_USFS8707","Soil_Type_USFS8708","Soil_Type_USFS8771","Soil_Type_USFS8772","Soil_Type_USFS8776","Cover_Type"]
print(data.shape)
display(data)
# display(data.head(10).T)

(581012, 55)


,Elevation,Aspect,Slope,HDH,VDH,HDR,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,HDFP,...,Soil_Type_USFS7756,Soil_Type_USFS7757,Soil_Type_USFS7790,Soil_Type_USFS8703,Soil_Type_USFS8707,Soil_Type_USFS8708,Soil_Type_USFS8771,Soil_Type_USFS8772,Soil_Type_USFS8776,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,3
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,3
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,3
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,3


In [19]:
def dataframe_to_dataset(dataframe:pd.DataFrame) -> tf.data.Dataset:
    dataframe = dataframe.copy()
    labels = dataframe.pop("Cover_Type")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))

    return ds

In [20]:
df = data.sample(frac=0.1, random_state=1337)
train_df = df.sample(frac=0.8, random_state=1337)
val_df = df.drop(train_df.index)
test_df = data.drop(df.index)
print(df.shape, train_df.shape ,val_df.shape ,test_df.shape)

(58101, 55) (46481, 55) (11620, 55) (522911, 55)


In [30]:
train_ds = dataframe_to_dataset(train_df)
val_ds = dataframe_to_dataset(val_df)
test_ds = dataframe_to_dataset(test_df)

In [57]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

Input: {'Elevation': <tf.Tensor: shape=(), dtype=int64, numpy=2205>, 'Aspect': <tf.Tensor: shape=(), dtype=int64, numpy=272>, 'Slope': <tf.Tensor: shape=(), dtype=int64, numpy=20>, 'HDH': <tf.Tensor: shape=(), dtype=int64, numpy=242>, 'VDH': <tf.Tensor: shape=(), dtype=int64, numpy=128>, 'HDR': <tf.Tensor: shape=(), dtype=int64, numpy=361>, 'Hillshade_9am': <tf.Tensor: shape=(), dtype=int64, numpy=162>, 'Hillshade_Noon': <tf.Tensor: shape=(), dtype=int64, numpy=241>, 'Hillshade_3pm': <tf.Tensor: shape=(), dtype=int64, numpy=216>, 'HDFP': <tf.Tensor: shape=(), dtype=int64, numpy=638>, 'Rawah_Wilderness_Area': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'Neota_Wilderness_Area': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'Comanche_Peak_Wilderness_Area': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'Cache_la_Poudre_Wilderness_Area': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'Soil_Type_USFS2702': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'Soil_Type_USFS2703': <tf.Tensor: shape=(), 

In [58]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)
test_ds = val_ds.batch(32)